In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash

C:\Users\NamNTBK\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
print(f'Max Payload: {max_payload}\nMin Payload: {min_payload}')

Max Payload: 9600.0
Min Payload: 0.0


In [4]:
spacex_df.drop(['Unnamed: 0'], axis=1, inplace=True)
spacex_df.head()

,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [5]:
# Create a dash application
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', options=[{'label':'All Sites', 'value':'OPT0'},
                                                                          {'label':'CCAFS LC-40','value':'OPT1'},
                                                                          {'label':'VAFB SLC-4E','value':'OPT2'},
                                                                          {'label':'KSC LC-39A','value':'OPT3'},
                                                                          {'label':'CCAFS SLC-40','value':'OPT4'}],
                                             placeholder='Select a launch site here',
                                             searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value')
             )
def get_graph(selected_site): 
    launch_sites = ['CCAFS LC-40','VAFB SLC-4E','KSC LC-39A','CCAFS SLC-40']
    # group the success rate by each launch site
    success_rate = spacex_df[['Launch Site', 'class']].groupby(['Launch Site']).mean()
    success_rate['fail_rate'] = 1 - success_rate['class']
    success_rate = success_rate.transpose()
    success_rate.index = ['0 - Fail','1 - Success']

    # create pie chart based on selected launch site
    if selected_site=='OPT0':
        success_rate = spacex_df[['Launch Site', 'class']].groupby(['Launch Site'], as_index=False).sum()
        fig = px.pie(success_rate, values='class', names='Launch Site', title='Total Success Launches By Site')
    else: 
        for i in range(1,5):
            if selected_site==f'OPT{i}':
                fig = px.pie(success_rate, values=launch_sites[i-1], names=success_rate.index, 
                             title='Total Success Launches for {}'.format(launch_sites[i-1]))
        
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'), 
            [Input(component_id='site-dropdown', component_property='value'), 
            Input(component_id="payload-slider", component_property="value")],
                
            )
def update_output(selected_site, slider_range):
    launch_sites = ['CCAFS LC-40','VAFB SLC-4E','KSC LC-39A','CCAFS SLC-40']
    low, high = slider_range
    mask = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    
    if selected_site=='OPT0':
        scatter_fig = px.scatter(spacex_df[mask], x='Payload Mass (kg)', y='class', color='Booster Version Category',
                                 width=1500, height=400, title='Correlation between Payload and Success for All Sites')        
    else: 
        for i in range(1,5):
            if selected_site==f'OPT{i}':
                mask1 = (spacex_df['Launch Site']==launch_sites[i-1])
                scatter_fig = px.scatter(spacex_df[mask&mask1], x='Payload Mass (kg)', y='class', color='Booster Version Category',
                                 width=1500, height=400, title=f'Correlation between Payload and Success for {launch_sites[i-1]}')
    return scatter_fig


In [77]:
px.scatter(spacex_df[spacex_df['Launch Site']=='CCAFS LC-40'], x='Payload Mass (kg)', y='class', color='Booster Version Category', width=1000, height=400)

In [6]:
# Run the app
if __name__ == '__main__':
    app.run_server(host="localhost",debug=True)

Dash app running on http://localhost:8050/
